In [38]:
import pandas as pd
import numpy as np
import os
import re
from nltk.tokenize import sent_tokenize

In [39]:
languages = {
    0: 'Danish', 1: 'German',
    2: 'Greek', 3: 'English',
    4: 'Spanish', 5: 'Finnish',
    6: 'French', 7: 'Italian',
    8: 'Dutch', 9: 'Portuguese',
    10: 'Swedish', 11: 'Bulgarian',
    12: 'Czech', 13: 'Estonian',
    14: 'Hungarian', 15: 'Lithuanian',
    16: 'Latvian', 17: 'Polish',
    18: 'Romanian', 19: 'Slovak',
    20: 'Slovenian'
}

In [40]:
def extract_language(language):
    with open(os.getcwd() + '/dataset/' + language +".txt") as outfile:
        lang = outfile.read()
    return lang.lower()

def clean(language):
    pattern = r'<(!?).*>'    
    
    language = re.sub(pattern, '', language)
    
    language = ''.join([i for i in language if not i.isdigit()])
    language = ''.join([i for i in language if i not in "(){}[]\n,'"])
    
    language = sent_tokenize(language)
    language = [i for i in language if len(i)> 4]
    return language
    
def stack(sentences, langauge_id, language):
    length = len(sentences)
    
    target = [langauge_id] * length
    lang = [language] * length
    
    df = pd.DataFrame(np.c_[sentences, target, lang], columns=['Sentences','Target', 'Language'])
    return df

def shuffle(dataframe):
    return dataframe.sample(frac=1).reset_index(drop=True)

def preprocess():
    data = pd.DataFrame([])
    for code,language in languages.items():
        extracted = extract_language(language.lower())
        cleaned = clean(extracted)
        dataframe = stack(cleaned, code, language)
        
        data = data.append(dataframe, ignore_index=True)
    data = shuffle(data)
    data['Target'] = data['Target'].astype(int)
    return data

In [59]:
def total_lines():
    sum = 0
    for code, lang in languages.items():
        extracted = extract_language(lang.lower())
        cleaned = clean(extracted)
        sum += len(cleaned)
    return sum

In [60]:
total_lines()

11988

In [61]:
data = preprocess()

In [62]:
data

,Sentences,Target,Language
0,- gospod predsednik zahvaljujem se parlamentu ...,20,Slovenian
1,não têm de momento capacidade para continuar a...,9,Portuguese
2,zasadnutí konferencie osn o obchode a rozvoji ...,19,Slovak
3,to není nejlepší vizitka evropského zemědělstv...,12,Czech
4,je suis sûre que certains dentre vous au moins...,6,French
...,...,...,...
11983,είναι ιδιαιτέρως ανησυχητικό το γεγονός ότι αν...,2,Greek
11984,można wiele uczynić aby utrzymać impet niedawn...,17,Polish
11985,dit geeft geen al te best beeld van de europes...,8,Dutch
11986,holms betænkning om handel med råmaterialer og...,0,Danish


In [63]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [64]:
data['Target'].max()

20

In [65]:
#y = tf.keras.utils.to_categorical(data['Target'], num_classes=21)
y = data['Target']
y

0        20
1         9
2        19
3        12
4         6
         ..
11983     2
11984    17
11985     8
11986     0
11987    10
Name: Target, Length: 11988, dtype: int64

In [66]:
tok = tf.keras.preprocessing.text.Tokenizer()

In [67]:
tok.fit_on_texts(data['Sentences'])

In [68]:
texts = tok.texts_to_sequences(data['Sentences'])

In [69]:
vocab = len(tok.word_index) + 1
vocab

73244

In [70]:
pad = tf.keras.preprocessing.sequence.pad_sequences(texts,maxlen=(60))

In [81]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab,
                              output_dim=64,
                             input_length=60),
    #tf.keras.layers.LSTM(200),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(21, activation=tf.nn.softmax)
])

In [82]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 64)            4687616   
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 64)            256       
_________________________________________________________________
flatten (Flatten)            (None, 3840)              0         
_________________________________________________________________
dense (Dense)                (None, 21)                80661     
Total params: 4,768,533
Trainable params: 4,768,405
Non-trainable params: 128
_________________________________________________________________


In [83]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
X_train, X_test, y_train, y_test = train_test_split(pad, np.array(y), test_size=0.1, random_state=42)

In [86]:
model.fit(X_train,y_train,epochs=100, batch_size=512, validation_data=(X_test, y_test),
          callbacks=[tf.keras.callbacks.EarlyStopping(),
                     tf.keras.callbacks.TensorBoard(log_dir='./graph', write_graph=True, write_images=True)])

Train on 10789 samples, validate on 1199 samples
Epoch 1/100
10789/10789 [==============================] - 2s 227us/sample - loss: 2.7890 - accuracy: 0.1921 - val_loss: 2.9795 - val_accuracy: 0.3812
Epoch 2/100
10789/10789 [==============================] - 2s 186us/sample - loss: 0.7266 - accuracy: 0.9188 - val_loss: 2.8667 - val_accuracy: 0.7306
Epoch 3/100
10789/10789 [==============================] - 2s 188us/sample - loss: 0.2081 - accuracy: 0.9902 - val_loss: 2.7689 - val_accuracy: 0.8382
Epoch 4/100
10789/10789 [==============================] - 2s 190us/sample - loss: 0.0879 - accuracy: 0.9974 - val_loss: 2.6955 - val_accuracy: 0.8699
Epoch 5/100
10789/10789 [==============================] - 2s 185us/sample - loss: 0.0472 - accuracy: 0.9987 - val_loss: 2.6285 - val_accuracy: 0.9008
Epoch 6/100
10789/10789 [==============================] - 2s 185us/sample - loss: 0.0288 - accuracy: 0.9994 - val_loss: 2.5620 - val_accuracy: 0.9016
Epoch 7/100
10789/10789 [====================

10789/10789 [==============================] - 2s 181us/sample - loss: 8.1061e-04 - accuracy: 0.9995 - val_loss: 0.2023 - val_accuracy: 0.9450
Epoch 55/100
10789/10789 [==============================] - 2s 176us/sample - loss: 7.9952e-04 - accuracy: 0.9995 - val_loss: 0.2008 - val_accuracy: 0.9475
Epoch 56/100
10789/10789 [==============================] - 2s 172us/sample - loss: 7.5442e-04 - accuracy: 0.9996 - val_loss: 0.2007 - val_accuracy: 0.9458
Epoch 57/100
10789/10789 [==============================] - 2s 173us/sample - loss: 7.3751e-04 - accuracy: 0.9996 - val_loss: 0.1992 - val_accuracy: 0.9475
Epoch 58/100
10789/10789 [==============================] - 2s 174us/sample - loss: 7.5943e-04 - accuracy: 0.9995 - val_loss: 0.1985 - val_accuracy: 0.9458
Epoch 59/100
10789/10789 [==============================] - 2s 173us/sample - loss: 7.1516e-04 - accuracy: 0.9997 - val_loss: 0.1983 - val_accuracy: 0.9466
Epoch 60/100
10789/10789 [==============================] - 2s 172us/sample -

In [87]:
model.save('model.h5')

In [88]:
import pickle

In [89]:
with open('tokenizer.json', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [90]:
with open('tokenizer.json', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [91]:
## Just testing :) 

In [96]:
test_text = ['hello world my name is something']

In [97]:
text = tf.keras.preprocessing.sequence.pad_sequences(tok.texts_to_sequences(test_text), maxlen=60)

In [98]:
languages.get(model.predict_classes(text)[0])

'English'

In [99]:
tokenizer.texts_to_sequences(test_text)

[[2313, 806, 3938, 31, 8080]]

In [102]:
text = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(test_text), maxlen=60)

In [103]:
languages.get(model.predict_classes(text)[0])

'English'